# IMPORTS

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score



## LEITURA DOS DADOS

In [2]:
df = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Entrega/1_DF_CN-MCI_1_After_Pre_Processamento.csv') 
df_test = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Entrega/1_DF_Test_1_After_Pre_Processamento.csv')


label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
    'CN-MCI' : 4
}



# Tratamento

### Correlação Entre Colunas 

In [3]:
def remove_highly_correlated_features(data, target, threshold):
    """
    Remove features altamente correlacionadas, mantendo a que tem maior correlação com o target.

    Args:
        data (pd.DataFrame): DataFrame contendo as features.
        target (pd.Series): Coluna com os valores do target.
        threshold (float): Valor de correlação acima do qual consideramos features altamente correlacionadas.

    Returns:
        pd.DataFrame: DataFrame com as colunas altamente correlacionadas removidas.
        set: Conjunto das colunas removidas.
    """
    # Compute the correlation matrix
    corr_matrix = data.corr().abs()
    
    # Compute correlation of each feature with the target
    target_corr = data.corrwith(data['Transition']).abs()
    
    # Identify columns to remove based on correlation
    to_drop = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] >= threshold:
                col_i = corr_matrix.columns[i]
                col_j = corr_matrix.columns[j]
                
                # Remove the column with lower correlation with the target
                if target_corr[col_i] < target_corr[col_j]:
                    to_drop.add(col_i)
                else:
                    to_drop.add(col_j)

    # Drop the identified columns
    data_final = data.drop(columns=to_drop)

    return data_final, to_drop


### Desvio Padrão

In [4]:


#def remove_low_variance_features(data, threshold=0.01):
#    # Remove low variance features
#    selector = VarianceThreshold(threshold=threshold)
#    low_variance_data = selector.fit_transform(data.select_dtypes(include=['float64', 'int64']))
#
#    # Get the columns to keep
#    retained_columns = data.select_dtypes(include=['float64', 'int64']).columns[selector.get_support(indices=True)]
#
#    # Create a DataFrame with the retained columns
#    data_reduced = pd.DataFrame(low_variance_data, columns=retained_columns)
#
#    return data_reduced
#print(df.shape)
##df = remove_low_variance_features(df)
#print(df.shape)


In [5]:
# *!  df.to_csv('DF_1_After_Pre_Processamento.csv', index=False)
# *! ## df_test.to_csv('DF_Test_1_After_Pre_Processamento.csv', index=False)


### Funcao Axuliar: Matrix de Confusão

In [6]:
def plot_confusion_matrix_with_labels(confusion_matrix, label_mapping):
    # Criar um mapeamento inverso
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    
    # Aplicar o mapeamento inverso na matriz de confusão
    cm_with_labels = np.zeros_like(confusion_matrix, dtype=object)
    for i in range(confusion_matrix.shape[0]):
        for j in range(confusion_matrix.shape[1]):
            cm_with_labels[i, j] = f"{reverse_label_mapping[i]} (Pred: {reverse_label_mapping[j]})"

    # Criar um DataFrame para facilitar a visualização
    df_cm = pd.DataFrame(confusion_matrix, index=reverse_label_mapping.values(), columns=reverse_label_mapping.values())
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix with Labels')
    plt.show()

# MODELOS

Função que testa todos os nossos modelos que estamos a analisar, consuante um certo dataset

In [7]:
# função que constrói os modelos, apresentando as métricas
def train_and_predict(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=34, stratify=y)
    modelos = [
     ExtraTreesClassifier(criterion='gini', max_depth=20, random_state=34),
    RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=34),
    GradientBoostingClassifier(
        learning_rate=0.1, n_estimators=100, random_state=34
    ),
   KNeighborsClassifier(
        n_neighbors=5, weights='uniform', algorithm='auto'
    ),
    SVC(
        kernel='rbf', random_state=34
    ),
    XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)
    ]
             

    reports = []
    f1_scores = [] 
    pred = []
    results = []
    cv_scores = [] 
    names = ['ExtraTrees', 'RandomForest', 'GradientBoosting', 'KNeighbors', 'SVC', 'XGBOOST']

    for i, model in enumerate(modelos):
        # Treina o modelo
        model.fit(X_train, y_train)



        # Faz previsões no conjunto de teste
        y_pred = model.predict(X_test)
        pred.append(y_pred)  # Armazena as previsões

        # Calcula e armazena o relatório de classificação
        report = classification_report(y_test, y_pred, zero_division=0)
        reports.append(report)

        # Calcula e armazena o F1 Macro Score
        f1 = f1_score(y_test, y_pred, average='macro')
        f1_scores.append(f1)

    for i, model in enumerate(modelos):        
        
        scores = cross_val_score(model, X, y, cv=5, scoring='f1_macro')
        cv_scores.append(scores)  
  
    

    # Exibe os relatórios e os F1 Scores lado a lado
    for i, report in enumerate(reports):
        print("Model {} ({}):\n{}".format(i + 1, names[i], report))
        print("F1_Macro: {:.5f}".format(f1_scores[i]))
        print("Mean CV F1_Macro: {:.5f}".format(cv_scores[i].mean()))
        print("=" * 50)
        results.append({'model': names[i],
            'f1_macro': f1_scores[i],
            'cv_f1_macro_mean':cv_scores[i].mean()})
    

    return results





In [8]:
#X = df.drop('Transition', axis=1)
#y = df['Transition']
#
#_ = train_and_predict(X,y)

In [9]:
thresholds = [1.1, 0.88, 0.89, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96]
all_results = []  # Lista para armazenar os resultados

for threshold in thresholds:
    print(f"Testing threshold: {threshold}\n\n")
    
    # Remover features altamente correlacionadas
    X_cleaned, to_drop = remove_highly_correlated_features(df, 'Transition', threshold)
    X_Limpo = X_cleaned.drop('Transition', axis=1)
    y = X_cleaned['Transition']
    print(f"Threshold: {threshold}, Features removed: {len(to_drop)}, Remaining features: {X_cleaned.shape[1]}")
    
    # Testar o dataset limpo com a função train_and_predict
    results = train_and_predict(X_Limpo, y)
    
    # Salvar os resultados do threshold atual
    all_results.append({
        'threshold': threshold,
        'features_removed': len(to_drop),
        'remaining_features': X_cleaned.shape[1],
        'results': results
    })
    print("\n\n\n ##########################################\n\n\n")

# Comparar e imprimir os resultados no final
print("\nSummary of Results:")
for res in all_results:
    print(f"Threshold: {res['threshold']}, Features removed: {res['features_removed']}, "
          f"Remaining features: {res['remaining_features']}, Results: {res['results']}")




Testing threshold: 1.1


Threshold: 1.1, Features removed: 0, Remaining features: 2014
Model 1 (ExtraTrees):
              precision    recall  f1-score   support

           0       0.57      0.71      0.63        24
           1       0.38      0.53      0.44        15
           2       0.06      0.06      0.06        17
           3       0.30      0.17      0.21        18
           4       0.00      0.00      0.00         3

    accuracy                           0.38        77
   macro avg       0.26      0.29      0.27        77
weighted avg       0.33      0.38      0.35        77

F1_Macro: 0.26979
Mean CV F1_Macro: 0.30025
Model 2 (RandomForest):
              precision    recall  f1-score   support

           0       0.57      0.71      0.63        24
           1       0.44      0.47      0.45        15
           2       0.10      0.12      0.11        17
           3       0.30      0.17      0.21        18
           4       0.00      0.00      0.00         3

    accu

| Modelo           | F1_Score (1.1)  | F1_Score (0.88) | F1_Score (0.89) | F1_Score (0.90) | F1_Score (0.91) | F1_Score (0.92) | F1_Score (0.93) | F1_Score (0.94) | F1_Score (0.95) | F1_Score (0.96) |
|------------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|
| **ExtraTrees**   | 0.2698          | 0.3046          | 0.2794          | 0.3526          | 0.2866          | 0.2846          | **0.3707**      | 0.2544          | 0.3356          | 0.3118          |
| **RandomForest** | 0.2802          | **0.3686**      | 0.3235          | 0.3422          | 0.3169          | 0.2633          | 0.3439          | 0.2816          | 0.2976          | 0.2685          |
| **GradientBoost**| 0.3046          | 0.3031          | **0.3637**      | 0.3152          | 0.3138          | 0.2973          | 0.3127          | 0.2923          | 0.2900          | 0.2971          |
| **KNeighbors**   | 0.2532          | 0.2600          | 0.2600          | 0.2444          | 0.2444          | 0.2444          | 0.2308          | 0.2308          | 0.2311          | 0.2311          |
| **SVC**          | 0.2548          | 0.2571          | 0.2571          | 0.2571          | 0.2571          | 0.2571          | 0.2571          | 0.2571          | 0.2571          | 0.2571          |
| **XGBOOST**      | 0.3150          | **0.3697**      | 0.3447          | 0.3448          | 0.3194          | 0.3393          | 0.3181          | 0.2812          | 0.2788          | 0.3068          |



In [10]:
thresholds = [1.1, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87]
all_results = []  # Lista para armazenar os resultados

for threshold in thresholds:
    print(f"Testing threshold: {threshold}\n\n")
    
    # Remover features altamente correlacionadas
    X_cleaned, to_drop = remove_highly_correlated_features(df, 'Transition', threshold)
    X_Limpo = X_cleaned.drop('Transition', axis=1)
    y = X_cleaned['Transition']
    print(f"Threshold: {threshold}, Features removed: {len(to_drop)}, Remaining features: {X_cleaned.shape[1]}")
    
    # Testar o dataset limpo com a função train_and_predict
    results = train_and_predict(X_Limpo, y)
    
    # Salvar os resultados do threshold atual
    all_results.append({
        'threshold': threshold,
        'features_removed': len(to_drop),
        'remaining_features': X_cleaned.shape[1],
        'results': results
    })
    print("\n\n\n ##########################################\n\n\n")

# Comparar e imprimir os resultados no final
print("\nSummary of Results:")
for res in all_results:
    print(f"Threshold: {res['threshold']}, Features removed: {res['features_removed']}, "
          f"Remaining features: {res['remaining_features']}, Results: {res['results']}")




Testing threshold: 1.1


Threshold: 1.1, Features removed: 0, Remaining features: 2014
Model 1 (ExtraTrees):
              precision    recall  f1-score   support

           0       0.57      0.71      0.63        24
           1       0.38      0.53      0.44        15
           2       0.06      0.06      0.06        17
           3       0.30      0.17      0.21        18
           4       0.00      0.00      0.00         3

    accuracy                           0.38        77
   macro avg       0.26      0.29      0.27        77
weighted avg       0.33      0.38      0.35        77

F1_Macro: 0.26979
Mean CV F1_Macro: 0.30025
Model 2 (RandomForest):
              precision    recall  f1-score   support

           0       0.57      0.71      0.63        24
           1       0.44      0.47      0.45        15
           2       0.10      0.12      0.11        17
           3       0.30      0.17      0.21        18
           4       0.00      0.00      0.00         3

    accu